In [1]:
import os
import json
import shutil

In [2]:
root_dir = '/home/kyuyeon/ssd/sandbox/__dataset/MUSIC'
vid_labels = dict()

In [3]:
with open(os.path.join(root_dir, 'MUSIC_solo_videos.json')) as json_file:
    music_json = json.load(json_file)
music_json = music_json['videos']

In [4]:
classes = sorted(music_json.keys())
print(classes)

['accordion', 'acoustic_guitar', 'cello', 'clarinet', 'erhu', 'flute', 'saxophone', 'trumpet', 'tuba', 'violin', 'xylophone']


In [5]:
for vid_class in classes:
    for vid_id in music_json[vid_class]:
        vid_labels[vid_id] = vid_class
print(len(vid_labels))

536


In [6]:
vid_exist_id = os.listdir(os.path.join(root_dir, 'png'))
remove_list = list()
for vid_id in vid_labels.keys():
    if vid_id not in vid_exist_id:
        remove_list.append(vid_id)
for rm_item in remove_list:
    vid_labels.pop(rm_item, None)
print(len(vid_labels))

493


In [7]:
from collections import Counter

counter = Counter(vid_labels.values())
counter = counter.most_common()
print(counter)

[('accordion', 57), ('cello', 56), ('acoustic_guitar', 55), ('erhu', 48), ('tuba', 48), ('violin', 46), ('flute', 43), ('xylophone', 40), ('trumpet', 39), ('clarinet', 38), ('saxophone', 23)]


In [8]:
s = sum([x[1] for x in counter])
s

493

In [9]:
n_pick = {x[0]: int(x[1] / 493 * 50) for x in counter}
n_pick

{'accordion': 5,
 'cello': 5,
 'acoustic_guitar': 5,
 'erhu': 4,
 'tuba': 4,
 'violin': 4,
 'flute': 4,
 'xylophone': 4,
 'trumpet': 3,
 'clarinet': 3,
 'saxophone': 2}

In [10]:
sum(n_pick.values())

43

In [11]:
music_json = dict()

for k, v in vid_labels.items():
    if v in music_json:
        music_json[v].append(k)
    else:
        music_json[v] = [k]
print(music_json.keys())

dict_keys(['accordion', 'acoustic_guitar', 'cello', 'clarinet', 'erhu', 'flute', 'saxophone', 'trumpet', 'tuba', 'violin', 'xylophone'])


In [12]:
import numpy.random as random

val_set_ids = dict()
for vid_class in music_json.keys():
    val_set_ids[vid_class] = random.choice(music_json[vid_class], size=n_pick[vid_class], replace=False).tolist()
sum([len(x) for x in val_set_ids.values()])

43

In [13]:
from copy import deepcopy

music_json_remain = deepcopy(music_json)
for vid_class in val_set_ids:
    for selected_id in val_set_ids[vid_class]:
        music_json_remain[vid_class].remove(selected_id)
sum([len(v) for _, v in music_json_remain.items()])

450

In [14]:
increment_list = list(n_pick.keys())[:7]
n_pick = {k: (v + 1) if k in increment_list else v for k, v in n_pick.items()}
sum(n_pick.values())

50

In [15]:
test_set_ids = dict()
for vid_class in music_json.keys():
    test_set_ids[vid_class] = random.choice(music_json_remain[vid_class], size=n_pick[vid_class], replace=False).tolist()
sum([len(x) for x in test_set_ids.values()])

50

In [16]:
music_json_remain_2 = deepcopy(music_json_remain)
for vid_class in test_set_ids:
    for selected_id in test_set_ids[vid_class]:
        music_json_remain_2[vid_class].remove(selected_id)
print(sum([len(v) for _, v in music_json_remain_2.items()]))
train_set_ids = dict()
for vid_class in music_json_remain_2.keys():
    train_set_ids[vid_class] = music_json_remain_2[vid_class]
sum([len(x) for x in train_set_ids.values()])

400


400

In [17]:
with open('val_split.json', 'w') as json_file:
    json.dump(val_set_ids, json_file)
with open('test_split.json', 'w') as json_file:
    json.dump(test_set_ids, json_file)
with open('train_split.json', 'w') as json_file:
    json.dump(train_set_ids, json_file)
